In [1]:
import numpy as np
import pandas as pd

import plotly
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

init_notebook_mode(connected=True)

In [2]:
total = [3779274,554867,5382780,2298739,30476517,4345321,2844358,757455,569480,16782417,7914681,
    1121794,1273151,9904838,5093409,2413764,2200585,3443650,3575930,1083273,4704671,5489864,
    7785662,4277949,2270533,4730561,821604,1444343,2312576,1084022,7026626,1599980,15694902,
    7971073,579621,9053374,2971579,3269157,10141022,852307,3919695,654810,5208482,20938557,2175134,
    506832,6600844,5759927,1446139,4512839,442832]

vec = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN',
    'IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
    'NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA',
    'WA','WV','WI','WY']
dic = {k:v for k,v in zip(vec, total)}

dframe = pd.read_csv('state_users.dat', sep=',', names=['state', 'cnt'])

In [3]:
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

code_states = {v:k for (k,v) in state_codes.items()}

In [4]:
## Divisions with states
new_england = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT']
mid_atlantic = ['NJ', 'NY', 'PA']
east_north_central = ['IN', 'IL', 'MI', 'OH', 'WI']
west_north_central = ['IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']
south_atlantic = ['DE', 'DC', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'WV']
east_south_central = ['AL', 'KY', 'MS', 'TN']
west_south_central = ['AR', 'LA', 'OK', 'TX']
mountain = ['AZ', 'CO', 'ID', 'NM', 'MT', 'UT', 'NV', 'WY']
pacific = ['CA', 'OR', 'WA','AK', 'HI']

states = new_england + mid_atlantic + east_north_central + west_north_central + south_atlantic + \
    east_south_central + west_south_central + mountain + pacific
    
dframe['status'] = dframe['state'].apply(lambda x: (str(x).startswith('region') and x[-2:] in states)) 
dframe['status1'] = dframe['state'].apply(lambda x: (str(x).startswith('region') and x[-2:] in code_states)) 
df = dframe[dframe.status==1]
data = dframe[dframe.status1==1]
df['code'] = df['state'].apply(lambda x: x[-2:])
data['code'] = data['state'].apply(lambda x: code_states[x[-2:]])

In [5]:
df = df.merge(data, on='code', how='inner')
df['cnt'] = df['cnt_x'] + df['cnt_y']
df['total'] = df['code'].apply(lambda x: dic[x])
df['pct'] = df['cnt'].astype(float)/df['total']
df['pct'] = np.where(df['pct']<=1.0, df['pct'], 1.0 )

In [7]:
#scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
#            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

scl = [[0.0, 'rgb(165,0,38)'], [0.1111111111111111, 'rgb(215,48,39)'], [0.2222222222222222, 'rgb(244,109,67)'],
        [0.3333333333333333, 'rgb(253,174,97)'], [0.4444444444444444, 'rgb(254,224,144)'], [0.5555555555555556, 'rgb(224,243,248)'],
        [0.6666666666666666, 'rgb(171,217,233)'],[0.7777777777777778, 'rgb(116,173,209)'], [0.8888888888888888, 'rgb(69,117,180)'],
        [1.0, 'rgb(49,54,149)']]

df['text'] = df['code'] + '<br>' + df['pct'].astype(str)

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['code'],
        z = df['pct'],
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Percentage")
        ) ]

layout = dict(
        title = 'Penetration by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
plot( fig, filename='us-map.html')

'file:///Users/peizhoul/Documents/Python/Dash/us-map.html'